# Analysis for Correlation: Correlation Heat Matrix

# 实验数据自动相关性分析（快速上手）
# —— 仅需 5 步 ——


1.  准备数据（Excel）
   - 表头全部用英文/数字/下划线（ASCII），不要中文或全角。
   - 第一列名必须是 index，行值用 I/II + CK/Cat/Glu/Gly（如 I-CK、II-Glu）。
   - 其余列都是数值型结果变量（可增减）。
   - 保存为 .xlsx 或 .xls

---

2. 环境准备
- 安装 Python
- cmd里: 
    - `pip install jupyter`
    - `pip install pandas matplotlib networkx openpyxl ipywidgets`
    - `jupyter lab` 
- 输入 `jupyter lab` 之后，jupynotebook 就被打开
- 点左上角加号 notebook创建新的notebook，或者文件-导入我这个notebook
---



3. 打开并配置脚本顶部参数（红色“只需修改本段”那块）
- MY_EXCEL_FILE = "你的数据.xlsx"
- SHOW_CORRELATIONS_ABOVE = 0.30      # 只显示 |r| ≥ 阈值 的连线
- TOP_CORRELATIONS_PER_TREATMENT = 6  # 每个处理保留前 k 个强相关
- MY_ANALYSIS_TITLE = "Treatment–Outcome Correlations"  # 图标题（英文保留）


---


4. 运行程序
- 终端：python your_script.py
- Jupyter：运行整份 Notebook
 - 终端/输出区会提示进度；若找不到文件，请检查路径与扩展名。

---

5. 看结果 & 出图
 - 会在同目录生成：<你的数据>_correlation_results.csv
 - 在 Jupyter 底部有两个中文控制面板：
     • “热图控制”：改配色/尺寸/角度 → 点“重绘热图”；点“保存热图 PNG”导出。
     • “网络图微调”：改标签距离/圆半径/阈值/Top 数/图例位置 → 点“重绘”；点“保存网络图 PNG”导出。
 - 图表字体固定为 Times New Roman；标题/图例保持英文，界面与提示为中文。


In [2]:
# -*- coding: utf-8 -*-
"""
╔════════════════════════════════════════════════════════════════════════════╗
║        实验数据自动相关性分析（Automatic Correlation Analysis）                ║
║        版本: 2.0 - 学生友好版                                                ║
╚════════════════════════════════════════════════════════════════════════════╝
"""

# =============================================================================
# 🔴 只需修改本段（学生看到的标题等）
# =============================================================================
MY_EXCEL_FILE = "gao_pca.xlsx"
SHOW_CORRELATIONS_ABOVE = 0.30
TOP_CORRELATIONS_PER_TREATMENT = 6
MY_ANALYSIS_TITLE = "Treatment–Outcome Correlations"
# 单因子“梯度”模式（可选）
FACTOR_HINT = None        # 例如 "Zn" / "Cu"；None 自动识别
LEVEL_ORDER = None        # 强制顺序，例如 ["Zn0","LZn","HZn"]；None 自动排序
MAX_LEVELS  = 50          # index 唯一水平上限（防误判）
INTERACTIVE_LEVEL_PROMPT = True   # 🆕 总是先问用户是否使用单因子梯度，并逐项确认顺序



# =============================================================================
# 🌏 Language / 语言
# =============================================================================
LOCALE = "zh"   # "zh" for Chinese, "en" for English

# Only the widgets/console are Chinese; plots stay English
UI_LOCALE = "zh"          # widgets + console
PLOTS_IN_CHINESE = False  # plots: keep English text
def t_ui(key, **kw):  # use this for widgets/prints
    return TR.get(UI_LOCALE, TR["zh"]).get(key, key).format(**kw)
def t_plot(key, **kw):    # use this for plot text
    return (TR["zh"] if PLOTS_IN_CHINESE else TR["en"]).get(key, key).format(**kw)


TR = {
    "en": {
        "welcome_banner": "\n╔════════════════════════════════════════════════════════════════════╗\n║   📊 WELCOME TO THE CORRELATION ANALYSIS TOOL                     ║\n╚════════════════════════════════════════════════════════════════════╝\n",
        "sample_title": "Expected Excel/CSV format (first rows look like this):",
        "legend_block": "Legend:\n- First 2 columns (SMS, PM; or SMS1, SMS2) = compost/material indicators (binary).\n- Next 4 columns (CK, Cat, Glu, Gly) = additives (binary).\n- Remaining columns = outcome variables (numeric measurements).\n",
        "using_file": "Using Excel file",
        "thr": "|r| threshold",
        "topk": "Top per treatment",
        "loaded_rows": "Loaded {n} rows from",
        "cleaning": "Cleaning data...",
        "removed_cols": "Removed {n} problematic column(s).",
        "cant_find": "ERROR: Cannot find the file",
        "current_folder": "Current folder",
        "excel_here": "Excel files here",
        "built_from_index": "Built design from index:",
        "materials": "Materials",
        "additives": "Additives",
        "outcomes": "Outcomes",
        "ask_I": "Enter material name for Treatment I [For example: {default_I}]: ",
        "ask_II": "Enter material name for Treatment II [For example: {default_II}]: ",
        "auto_detected": "Auto-detected design (from index):",
        "fallback_detected": "Auto-detected design:",
        "no_numeric": "ERROR: No numeric outcome columns detected.",
        "found_treatments": "Found {k} unique treatment combinations:",
        "design_shape": "Design matrix: {n} samples × {p} treatments",
        "calc_corr": "Calculating correlations with {m} outcomes...",
        "calc_done": "Correlations calculated.",
        "saved_to": "Results saved to",
        "hm_creating": "Creating heatmap...",
        "hm_done": "Heatmap displayed.",
        "net_creating": "Creating circular network visualization...",
        "no_edges": "No correlations found above {thr}",
        "net_done": "Pie network created with {E} edges.",
        "sig_header": "Significant correlations (|r| > {thr}):",
        "none": "None.",
        "start": "STARTING AUTOMATED CORRELATION ANALYSIS",
        "viz_header": "CREATING VISUALIZATIONS",
        "viz1": "VISUALIZATION 1: Correlation Heatmap",
        "viz2": "VISUALIZATION 2: Circular Network",
        "top_header": "TOP CORRELATIONS",
        "complete": "ANALYSIS COMPLETE!",
        "summary_data": "Data analyzed",
        "summary_design": "Design",
        "summary_outcomes": "Outcomes analyzed",
        "heatmap_controls": "Heatmap controls",
        "heatmap_colors": "Heatmap colors",
        "c1": "c1 (neg, -)",
        "c3": "c3 (mid)",
        "c5": "c5 (pos, +)",
        "size": "Size",
        "use_explicit": "Use explicit size",
        "width_in": "Width (in)",
        "height_in": "Height (in)",
        "display": "Display",
        "annotate": "Annotate cells",
        "center0": "Center at 0 (±vmax)",
        "xrot": "X rotation",
        "vmax_abs": "vmax_abs (optional)",
        "grid_minor": "Show minor grid",
        "redraw_heatmap": "Redraw Heatmap",
        "save": "Save",
        "filename": "Filename",
        "dpi": "DPI",
        "transparent": "Transparent",
        "save_hm": "Save Heatmap PNG",
        "drawing_hm": "Drawing correlation heatmap...",
        "net_controls_tip": "Use the controls below to fine-tune the network layout.",
        "shown": "Heatmap shown.",
        "network_tuning": "Network tuning",
        "label_dist": "Label distance",
        "circle_radius": "Circle radius",
        "edge_curv": "Edge curvature",
        "r_thresh": "|r| threshold",
        "top_per_trt": "Top per trt",
        "legend_loc": "Legend loc",
        "legend_cols": "legend cols",
        "legend_pad": "legend pad",
        "title_pad_frac": "title pad (frac)",
        "redraw": "Redraw",
        "save_net": "Save Network PNG",
        "drawing_net": "Drawing circular network...",
        "net_title": "Circular Network (|r| ≥ {thr}, top {k}/treatment)",
        "legend_trt": "Treatment",
        "legend_out": "Outcome",
        "legend_pos": "Positive correlation",
        "legend_neg": "Negative correlation",
        "colorbar": "Pearson Correlation",
    },
    "zh": {
        "welcome_banner": "\n╔════════════════════════════════════════════════════════════════════╗\n║   📊 欢迎使用相关性分析工具                                        ║\n╚════════════════════════════════════════════════════════════════════╝\n",
        "sample_title": "期望的 Excel/CSV 格式（前几行示例）：",
        "legend_block": "说明：\n- 前两列（如 SMS、PM 或 SMS1、SMS2）= 原料/材料指示（0/1）。\n- 接下来的四列（CK、Cat、Glu、Gly）= 添加剂指示（0/1）。\n- 其余列 = 各种测量的数值型结果变量。\n",
        "using_file": "使用的 Excel 文件",
        "thr": "|r| 阈值",
        "topk": "每个处理的 Top 数",
        "loaded_rows": "已从文件读取 {n} 行：",
        "cleaning": "正在清理数据…",
        "removed_cols": "已移除 {n} 个存在问题的列。",
        "cant_find": "错误：找不到文件",
        "current_folder": "当前文件夹",
        "excel_here": "此处的 Excel 文件",
        "built_from_index": "已根据索引/标签列构建设计矩阵：",
        "materials": "原料/材料",
        "additives": "添加剂",
        "outcomes": "结果变量",
        "ask_I": "请输入“处理 I”的材料名称（例如 {default_I}）：",
        "ask_II": "请输入“处理 II”的材料名称（例如 {default_II}）：",
        "auto_detected": "（索引解析）自动识别的设计：",
        "fallback_detected": "自动识别的设计：",
        "no_numeric": "错误：未检测到数值型结果列。",
        "found_treatments": "共发现 {k} 种处理组合：",
        "design_shape": "设计矩阵：{n} 个样本 × {p} 个处理",
        "calc_corr": "正在计算与 {m} 个结果变量的相关系数…",
        "calc_done": "相关计算完成。",
        "saved_to": "结果已保存至",
        "hm_creating": "正在绘制热图…",
        "hm_done": "热图已显示。",
        "no_edges": "没有相关系数高于阈值 {thr}",
        "net_done": "网络图已生成，连边数 {E}。",
        "sig_header": "显著相关（|r| > {thr}）：",
        "none": "无。",
        "start": "开始自动相关性分析",
        "viz_header": "开始创建可视化",
        "viz1": "可视化 1：相关性热图",
        "viz2": "可视化 2：环形网络图",
        "top_header": "Top 相关结果",
        "complete": "分析完成！",
        "summary_data": "数据文件",
        "summary_design": "设计规模",
        "summary_outcomes": "结果变量数量",
        "heatmap_controls": "热图控制",
        "heatmap_colors": "热图颜色",
        "c1": "c1（负向，-）",
        "c3": "c3（中性）",
        "c5": "c5（正向，+）",
        "size": "尺寸",
        "use_explicit": "使用固定尺寸",
        "width_in": "宽度（英寸）",
        "height_in": "高度（英寸）",
        "display": "显示",
        "annotate": "显示数值",
        "center0": "以 0 为中心（±vmax）",
        "xrot": "X 轴刻度旋转",
        "vmax_abs": "vmax_abs（可选）",
        "grid_minor": "显示网格",
        "redraw_heatmap": "重绘热图",
        "save": "保存",
        "filename": "文件名",
        "dpi": "DPI",
        "transparent": "透明背景",
        "save_hm": "保存热图 PNG",
        "net_controls_tip": "使用下面的滑块微调网络图布局。",
        "shown": "热图已显示。",
        "network_tuning": "此处微调。",
        "label_dist": "标签间距",
        "circle_radius": "圆半径",
        "edge_curv": "边曲率",
        "r_thresh": "相关阈值 |r|",
        "top_per_trt": "每处理 Top 数",
        "legend_loc": "图例位置",
        "legend_cols": "图例列数",
        "legend_pad": "图例偏移",
        "title_pad_frac": "标题位置（相对）",
        "redraw": "重绘",
        "save_net": "保存网络图 PNG",
        "drawing_net": "正在绘制环形网络图…",
        "net_title": "环形网络图（|r| ≥ {thr}，每处理取前 {k} 个）",
        "legend_trt": "处理",
        "legend_out": "结果",
        "legend_pos": "正相关",
        "legend_neg": "负相关",
        "colorbar": "皮尔逊相关系数",
    },
}


# add UI labels for font sliders
TR["en"].update({
    "tick fs": "Tick fontsize",
    "cell fs": "Cell value fontsize",
})
TR["zh"].update({
    "tick fs": "坐标刻度字体",
    "cell fs": "单元格数字字体",
})


def t(key, **kwargs):
    return TR.get(LOCALE, TR["zh"]).get(key, key).format(**kwargs)

# =============================================================================
# 🎨 视觉设置
# =============================================================================
NETWORK_SETTINGS = {
    "circle_radius": 1.3,
    "label_distance": 0.15,
    "figure_size": (12, 12),
    
    "treat_start_deg": -84.0,   # treatments arc start (default ≈ -90°+0.1rad)
    "treat_end_deg":    84.0,   # treatments arc end   (default ≈ +90°-0.1rad)
    "out_start_deg":    96.0,   # outcomes arc start   (default ≈ +90°+0.1rad)
    "out_end_deg":     264.0,   # outcomes arc end     (default ≈ +270°-0.1rad)

    "treatment_color": "#b87430",
    "outcome_color":   "#37b24d",
    "positive_edge":   "#e24a33",
    "negative_edge":   "#2aa198",
    "background":      "white",

    "node_base_size": 600,
    "node_size_increment": 100,

    "edge_base_width": 1.0,
    "edge_width_scale": 4.0,
    "edge_alpha": 0.7,
    "edge_curvature": 0.3,

    "label_fontsize": 11,
    "title_fontsize": 14,
    "legend_fontsize": 10,

    "show_legend": True,
    "legend_position": "lower center",
    "legend_anchor": (0.5, -0.04),
    "legend_ncol": 2,
    "show_title": True,
    "legend_pad": -0.07,
    "title_pad": 20,
    "title_pad_frac": -0.05,
}

HEATMAP_SETTINGS = {
    "figure_size_scale": 0.6,
    "show_values": True,
    "tick_fontsize": 9,   # NEW: axis tick label font size
    "value_fontsize": 7,  # numbers inside cells
    "colormap": ["#01756d", "#25c6b8", "#e8f9fa", "#fed5a9", "#fe8e2c"],
}


LABEL_MAP = {
    # Explicit names you mentioned
    "WSS": r"$C_{WSS}$",
    "CWSS": r"$C_{WSS}$",
    "CDOM": r"$C_{DOM}$",
    "DOM": r"$C_{DOM}$",
    "HA": r"$C_{HA}$",
    "CHA": r"$C_{HA}$",
    "HE": r"$C_{HE}$",
    "CHE": r"$C_{HE}$",
    "HU": r"$C_{Hu}$",     # note lowercase u
    "CHU": r"$C_{Hu}$",
    "LOGK": r"LogK",
    "LOG K": r"LogK",
    "P": r"$\mathrm{P_{2}O_{5}}$",
    "K": r"$\mathrm{K_{2}O}$",
    "E4/E6 of HA": r"$E_{4}/E_{6}\ \mathrm{(HA)}$",
    "E4/E6 (HA)": r"$E_{4}/E_{6}\ \mathrm{(HA)}$",
    "CHA/CFA": r"$C_{HA}/C_{FA}$",
    "Organic matter": r"Organic matter",
    "Maize yield": r"Maize yield",
    "HIX": r"HIX",
}

import re
KNOWN_C_SUFFIXES = {"WSS","DOM","HA","HE","HU","FA"}  # extend if needed

def latex_label(name: str) -> str:
    s = str(name).strip()
    if s in LABEL_MAP:
        return LABEL_MAP[s]

    u = s.upper().replace(" ", "")
    # Auto-map C+suffix (CWSS, CHE, CHU, etc.) → C_{...}
    m = re.fullmatch(r"C([A-Z]+)", u)
    if m and m.group(1) in KNOWN_C_SUFFIXES:
        suf = m.group(1)
        if suf == "HU":
            return r"$C_{Hu}$"    # keep lowercase u
        return rf"$C_{{{suf}}}$"

    # Bare suffix (WSS, HE, HU, DOM, HA, FA) → assume C_{...}
    if u in KNOWN_C_SUFFIXES:
        if u == "HU":
            return r"$C_{Hu}$"
        return rf"$C_{{{u}}}$"

    # Tolerate spacing in LOG K
    if u == "LOGK":
        return r"LogK"

    # (Optional) normalize simple E4/E6 patterns
    if re.fullmatch(r"E4/?E6(?:OFHA|\(HA\))?", u):
        return r"$E_{4}/E_{6}\ \mathrm{(HA)}$"

    # Default: unchanged
    return s


# =============================================================================
# Imports
# =============================================================================
import os, time, warnings, re
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm
from matplotlib.patches import FancyArrowPatch
from matplotlib.lines import Line2D
import networkx as nx

warnings.filterwarnings('ignore')

def _running_in_notebook():
    try:
        from IPython import get_ipython
        return get_ipython().__class__.__name__ == "ZMQInteractiveShell"
    except Exception:
        return False

if _running_in_notebook():
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML

# =============================================================================
# 示例格式（中文）
# =============================================================================
# —— 简洁中文上传指南（Zn 版本，打印用）——
TR["zh"].update({
    "sample_title": "上传前请按下述英文表头整理（仅 ASCII），示例：",
    "legend_block": (
        "上传规范：\n"
        "- 第一列表头必须为 index；行值仅能是 Zn0、LZn、HZn（锌的 3 个水平）。\n"
        "- 其它列为数值型结果变量（英文/数字/下划线，勿用中文或全角字符）。\n"
        "- 小数使用点号 .（如 7.83），不要逗号。\n"
        "- 可自行增删结果列；顺序不限。\n"
        "- 保存为 .xlsx/.xls 后上传。"
    )
})

SAMPLE_FORMAT = f"""
{t('sample_title')}

index | AlkaliN | AvailP | AvailK | SOM  | pH  | AvailZn | Yield
------+---------+--------+--------+------+-----+---------+-------
Zn0   |  98.4   |  19.4  |  172.4 | 14.8 | 6.11|  6.74   | 10257.1
LZn   |  65.7   |  33.0  |  189.7 | 15.0 | 5.34|  7.96   | 12426.8
HZn   |  85.5   |  20.7  |  176.7 | 13.8 | 5.12|  8.05   | 12522.5

{t('legend_block')}
"""



class AutoCorrelationAnalyzer:
    def __init__(self, excel_file, title, correlation_threshold, top_k,
                 network_settings=None, heatmap_settings=None):
        self.data_file = excel_file
        self.output_csv = excel_file.replace('.xlsx', '_correlation_results.csv').replace('.xls', '_correlation_results.csv')
        self.figure_title = title
        self.correlation_threshold = correlation_threshold
        self.top_k = top_k
        self.network_settings = network_settings or NETWORK_SETTINGS
        self.heatmap_settings = heatmap_settings or HEATMAP_SETTINGS
        self.color_scheme = self.heatmap_settings.get("colormap",
                            ["#01756d", "#25c6b8", "#e8f9fa", "#fed5a9", "#fe8e2c"])
        # Fonts: keep plots strictly in Times New Roman
        mpl.rcParams['font.family'] = 'Times New Roman'
        mpl.rcParams['mathtext.fontset'] = 'custom'
        mpl.rcParams['mathtext.rm'] = 'Times New Roman'
        mpl.rcParams['figure.constrained_layout.use'] = False


        self.df = None
        self.material_cols = None
        self.additive_cols = None
        self.outcome_cols = None
        self.design_matrix = None
        self.correlation_matrix = None
        self._last_heatmap = None
        self._last_network = None

    def _pretty(self, name: str) -> str:
        return latex_label(name)


    def check_file_exists(self):
        if not os.path.exists(self.data_file):
            print(f"\n❌ {t('cant_find')} '{self.data_file}'")
            print(f"\n📁 {t('current_folder')}: {os.getcwd()}")
            print(f"   {t('excel_here')}: {[f for f in os.listdir('.') if f.endswith(('.xlsx', '.xls'))]}")
            return False
        return True

    
    def _ask_levels_interactively(self, s: pd.Series, uniq: list, default_order: list):
        """
        交互式让用户确认是否采用“单因子梯度”，并按顺序输入水平。
        s: 规范化后的 index 序列
        uniq: 唯一标签列表
        default_order: 自动推断的默认顺序
        返回：pd.Series（类别型，带顺序）或 None（不使用梯度模式）
        """
        try:
            print("\n🔎 检测到可能的单因子标签：", ", ".join(map(str, default_order)))
            use = input("是否启用【单因子梯度】模式？(y/N)：").strip().lower()
        except Exception:
            use = "n"

        if use != "y":
            return None

        # 询问元素名称（可为空）
        try:
            hint = input("请输入元素名称/前缀（如 Zn/Cu；直接回车跳过）：").strip()
            if hint:
                # 动态更新全局提示，便于后续保存或再次运行
                globals()["FACTOR_HINT"] = hint
        except Exception:
            pass

        # 询问水平个数（默认=唯一标签数）
        try:
            n_default = len(default_order)
            n = input(f"共有几个水平？(默认 {n_default})：").strip()
            n = int(n) if n else n_default
        except Exception:
            n = len(default_order)

        # 逐项输入顺序（共 n 次），回车采用默认
        chosen = []
        for i in range(n):
            d = default_order[i] if i < len(default_order) else (default_order[-1] if default_order else "")
            try:
                val = input(f"第 {i+1} 个水平名称（回车采用默认 {d}）：").strip() or str(d)
            except Exception:
                val = str(d)
            chosen.append(val)

        # 仅保留数据中真实存在的水平，避免写错导致全空
        exist = set(map(str, uniq))
        final_order = [x for x in chosen if x in exist]
        if not final_order:               # 全都不在数据里，退回默认
            final_order = [x for x in default_order if x in exist] or list(exist)

        print("✅ 使用顺序：", " < " .join(final_order))
        return pd.Categorical(s, categories=final_order, ordered=True).astype(str)

    
    def _maybe_single_factor_from_index(self, labels, factor_hint=None):
        """
        labels: pd.Series[str]（已标准化的 index 列）
        返回：pd.Series[str] 的 Treatment（按有序分类排好序），或 None 表示不适用
        """
        s = labels.fillna("").astype(str).str.strip()
        if s.eq("").all():
            return None

        # 若像 I-/II- 这种组合很多，交给旧逻辑
        if (s.str.contains(r"\bI{1,3}\s*[-—－]").mean() > 0.4) or (s.str.contains(r"-").mean() > 0.6):
            return None

        # 有前缀提示（如 'Zn'/'Cu'），优先用“前缀+数字”抽取并按数值排序
        if factor_hint:
            import re
            pat = re.compile(rf"^{re.escape(factor_hint)}[\s_:-]*([+-]?\d+(?:\.\d+)?)", re.I)
            num = s.str.extract(pat, expand=False)
            if num.notna().any():
                vals = pd.to_numeric(num, errors="coerce")
                order = (pd.Series(vals).groupby(s, sort=False).mean().sort_values(kind="mergesort"))
                cats = list(order.index)
                # 🆕 先给用户机会自定义顺序
                if INTERACTIVE_LEVEL_PROMPT:
                    asked = self._ask_levels_interactively(s, uniq=list(pd.unique(s)), default_order=cats)
                    if asked is not None:
                        return asked
                return pd.Categorical(s, categories=cats, ordered=True).astype(str)

        # 无提示：自动类别化（上限 MAX_LEVELS）
        uniq = list(pd.unique(s))
        if len(uniq) == 0 or len(uniq) > MAX_LEVELS:
            return None

        # 能提取到数字就按数字；否则按字母
        nums = pd.to_numeric(s.str.extract(r"([+-]?\d+(?:\.\d+)?)", expand=False), errors="coerce")
        if nums.notna().sum() >= len(uniq) // 2:
            key = nums.groupby(s, sort=False).mean().sort_values(kind="mergesort")
            cat_order = list(key.index)
        else:
            cat_order = sorted(set(uniq), key=lambda x: (x.lower(), x))

        # 若给了 LEVEL_ORDER，优先用
        if LEVEL_ORDER:
            pref = [c for c in LEVEL_ORDER if c in set(uniq)]
            others = [c for c in cat_order if c not in set(pref)]
            cat_order = pref + others

        # 🆕 交互式逐项确认（会问 len 水平次）
        if INTERACTIVE_LEVEL_PROMPT:
            asked = self._ask_levels_interactively(s, uniq=uniq, default_order=cat_order)
            if asked is not None:
                return asked

        return pd.Categorical(s, categories=cat_order, ordered=True).astype(str)



    # --- Build design from labels like I-CK ---
    def _derive_design_from_index(self) -> bool:
        
        # in _derive_design_from_index()
        cand = [c for c in self.df.columns
                if str(c).strip().lower() in ("index","treatment","treatments","group","sample")]
        idx_col = cand[0] if cand else None
        # after: idx_col = ... (once it's known)
        self.df = self.df.dropna(axis=1, how='all').dropna(axis=0, how='all')
        for c in self.df.columns:
            if c != idx_col:
                self.df[c] = pd.to_numeric(self.df[c], errors="coerce")

        
        
        if idx_col is None:
            c0 = self.df.columns[0]
            if not pd.api.types.is_object_dtype(self.df[c0]):
                return False
            idx_col = c0

        roman_map = {"Ⅰ":"I","Ⅱ":"II","Ⅲ":"III","Ⅳ":"IV","Ⅴ":"V","Ⅵ":"VI","Ⅶ":"VII","Ⅷ":"VIII","Ⅸ":"IX","Ⅹ":"X",
                     "ⅰ":"I","ⅱ":"II","ⅲ":"III","ⅳ":"IV","ⅴ":"V","ⅵ":"VI","ⅶ":"VII","ⅷ":"VIII","ⅸ":"IX","ⅹ":"X"}
        def _norm(s: str) -> str:
            s = str(s)
            for k,v in roman_map.items():
                s = s.replace(k, v)
            s = s.replace("–","-").replace("—","-").replace("−","-")
            s = re.sub(r"\s*-\s*", "-", s)
            return s.strip()

        labels = self.df[idx_col].astype(str).map(_norm)
        # ===== 单因子“梯度”通用识别（任意元素/任意水平）=====

        treat_series = self._maybe_single_factor_from_index(labels, factor_hint=FACTOR_HINT)
        if treat_series is not None:
            self.df["Treatment"] = treat_series
            self.material_cols = []       # 单因子模式不用材料/添加剂
            self.additive_cols = []
            self._index_label_col = idx_col
            print("\n🧩 检测到单因子梯度设计：", list(pd.Categorical(treat_series).categories))
            return True


        parts = labels.str.extract(r"^\s*([A-Za-z]+|[IVX]+)\s*-\s*(.*)$", expand=True)
        if parts.isna().any().any():
            return False

        head = parts[0].str.upper().str.strip()
        tail = parts[1].fillna("").astype(str).str.strip()

        if not {"I","II"} & set(head.unique()):
            return False

        try:
            name_I  = input(t("ask_I", default_I="SMS")).strip() or "SMS"
            name_II = input(t("ask_II", default_II="PM")).strip()  or "PM"
        except Exception:
            name_I, name_II = "SMS", "PM"

        for col in [name_I, name_II, "CK", "Cat", "Glu", "Gly"]:
            if col not in self.df.columns:
                self.df[col] = 0

        self.df.loc[head=="I",  name_I]  = 1
        self.df.loc[head=="II", name_II] = 1

        def _map_add(tok: str):
            t0 = str(tok).strip().lower()
            if t0 in ("ck","control","ctrl"): return "CK"
            if t0.startswith("cat"): return "Cat"
            if t0.startswith("glu"): return "Glu"
            if t0.startswith("gly"): return "Gly"
            return None

        for i, s in tail.items():
            if not s:
                self.df.at[i, "CK"] = 1
                continue
            tokens = [p for p in re.split(r"[+,&/ ]+", s) if p]
            matched = False
            for tok in tokens:
                col = _map_add(tok)
                if col:
                    self.df.at[i, col] = 1; matched = True
            if not matched:
                self.df.at[i, "CK"] = 1

        self.material_cols = [name_I, name_II]
        add_candidates = ["CK","Cat","Glu","Gly"]
        self.additive_cols = [c for c in add_candidates if self.df[c].sum() > 0] or ["CK"]
        self._index_label_col = idx_col

        print("\n🧩 " + t("built_from_index"))
        print(f"  {t('materials')}: {self.material_cols}")
        print(f"  {t('additives')}: {self.additive_cols}")
        return True

    def load_and_detect(self):
        if not self.check_file_exists(): return None
        try:
            self.df = pd.read_excel(self.data_file)
        except Exception as e:
            print(f"\n❌ {e}")
            return None

        print(f"\n✅ {t('loaded_rows', n=len(self.df))} '{self.data_file}'")
        print("\n🧹 " + t("cleaning"))
        self._clean_column_names()

        if self._derive_design_from_index():
            non_outcome = set(self.material_cols + self.additive_cols)
            non_outcome.add(getattr(self, "_index_label_col", ""))
            self.outcome_cols = [c for c in self.df.columns
                                 if c not in non_outcome and pd.api.types.is_numeric_dtype(self.df[c])]
            print("\n🔍 " + t("auto_detected"))
            print(f"  📦 {t('materials')}: {self.material_cols}")
            print(f"  🧪 {t('additives')}: {self.additive_cols}")
            print(f"  📊 {t('outcomes')}: {self.outcome_cols[:5]}{' ...' if len(self.outcome_cols)>5 else ''}")
            if not self.outcome_cols:
                print("\n❌ " + t("no_numeric")); return None
            return self

        # fallback (unchanged logic, Chinese prints)
        all_cols = list(self.df.columns); start_idx = 0
        if 'index' in [c.lower() for c in all_cols[:2]]: start_idx = 1
        elif all_cols and str(all_cols[0]).startswith('Unnamed'): start_idx = 1

        binary_cols, numeric_cols = [], []
        for col in all_cols[start_idx:]:
            if pd.api.types.is_numeric_dtype(self.df[col]):
                vals = set(self.df[col].dropna().unique())
                (binary_cols if vals.issubset({0,1,0.0,1.0}) else numeric_cols).append(col)

        if len(binary_cols) >= 2:
            self.material_cols = binary_cols[:2]; self.additive_cols = binary_cols[2:6]
        else:
            self.material_cols = all_cols[start_idx:start_idx+2]
            self.additive_cols = all_cols[start_idx+2:start_idx+6]

        self.outcome_cols = [c for c in numeric_cols if c not in (self.material_cols + self.additive_cols)]

        print("\n🔍 " + t("fallback_detected"))
        print(f"  📦 {t('materials')}: {self.material_cols}")
        print(f"  🧪 {t('additives')}: {self.additive_cols}")
        print(f"  📊 {t('outcomes')}: {self.outcome_cols[:5]}{' ...' if len(self.outcome_cols)>5 else ''}")
        if not self.outcome_cols:
            print("\n❌ " + t("no_numeric")); return None
        return self

    def _clean_column_names(self):
        cols_to_drop = []
        for col in self.df.columns:
            s = str(col)
            if any(p in s for p in ['(a+b+c)/d', '（a+b+c)/d', 'a+b+c', 'a＋b＋c']):
                cols_to_drop.append(col)
        if cols_to_drop:
            self.df = self.df.drop(columns=cols_to_drop)
            print(f"  {t('removed_cols', n=len(cols_to_drop))}")

        repl = {'（': '(', '）': ')', '△': 'Δ', '／': '/', '：': ':',
                '，': ',', '。': '.', '＋': '+', '－': '-', '×': 'x'}
        clean_cols = []
        for col in self.df.columns:
            t0 = str(col)
            for a,b in repl.items():
                t0 = t0.replace(a,b)
            t0 = ''.join(ch if ord(ch)<128 else '_' for ch in t0)
            clean_cols.append(t0)
        self.df.columns = clean_cols

    def create_treatment_labels(self):
        def _get_material_label(row):
            if not self.material_cols: return "Unknown"
            act = [c for c in self.material_cols if row.get(c,0)>0]
            return "+".join(act) if act else "No-material"

        def _get_additive_label(row):
            if not self.additive_cols: return "None"
            for c in self.additive_cols:
                if 'ck' in c.lower() and row.get(c,0)>0:
                    return "Control"
            act = [c for c in self.additive_cols if 'ck' not in c.lower() and row.get(c,0)>0]
            return "+".join(act) if act else "Control"

        labels = []
        for _, row in self.df.iterrows():
            labels.append(f"{_get_material_label(row)} + {_get_additive_label(row)}")
        self.df['Treatment'] = labels

        uni = sorted(set(labels))
        print(f"\n📊 {t('found_treatments', k=len(uni))}")
        for i, tt in enumerate(uni, 1):
            print(f"   {i:2d}. {tt:<40} (n={(self.df['Treatment']==tt).sum()})")
        return uni

    def prepare_design_matrix(self):
        # 如果已经有 Treatment 列（例如 Zn 模式），直接用它
        if "Treatment" in self.df.columns:
            treatments = list(pd.Categorical(self.df["Treatment"]).categories)
            if not treatments:  # 兜底
                treatments = sorted(self.df["Treatment"].unique())
        else:
            treatments = self.create_treatment_labels()

        design = {t_: (self.df["Treatment"] == t_).astype(int) for t_ in treatments}
        self.design_matrix = pd.DataFrame(design, index=self.df.index)
        print(f"\n✅ {t('design_shape', n=self.design_matrix.shape[0], p=self.design_matrix.shape[1])}")
        return self


    def calculate_correlations(self):
        print(f"\n📈 {t('calc_corr', m=len(self.outcome_cols))}")
        X = self.df[self.outcome_cols].copy()
        corr = pd.concat([self.design_matrix, X], axis=1).corr(method='pearson')
        self.correlation_matrix = corr.loc[self.design_matrix.columns, self.outcome_cols].copy()
        print("✅ " + t("calc_done"))
        return self

    def save_results(self):
        with open(self.output_csv, 'w', encoding='utf-8') as f:
            f.write("# Correlation Analysis Results / 相关性分析结果\n")
            f.write(f"# Data file / 数据文件: {self.data_file}\n")
            f.write(f"# {t('materials')}: {', '.join(self.material_cols)}\n")
            f.write(f"# {t('additives')}: {', '.join(self.additive_cols)}\n")
            f.write(f"# {t('outcomes')}: {len(self.outcome_cols)}\n#\n")
        self.correlation_matrix.to_csv(self.output_csv, mode='a', encoding='utf-8')
        print(f"\n💾 {t('saved_to')} : {self.output_csv}")
        return self

    def plot_heatmap(self):
        # 1) Guard: matrix must exist and be non-empty
        if self.correlation_matrix is None or self.correlation_matrix.empty:
            # try to compute once if missing
            try:
                if self.design_matrix is not None and self.outcome_cols:
                    self.calculate_correlations()
            except Exception:
                pass
        if self.correlation_matrix is None or self.correlation_matrix.empty:
            print("⚠️ 无可绘制的相关矩阵（为空或尚未计算）。")
            return

        print("\n🎨 " + t("hm_creating"))
        s = self.heatmap_settings or {}

        # 2) Safe rows/cols
        rows = list(self.correlation_matrix.index)
        cols = list(self.correlation_matrix.columns)

        # 3) Safe colormap
        default_cm = ["#01756d", "#25c6b8", "#e8f9fa", "#fed5a9", "#fe8e2c"]
        colors = s.get("colormap")
        if not isinstance(colors, (list, tuple)) or len(colors) < 2:
            colors = default_cm[:]
        # normalize to exactly 5 anchors
        if len(colors) < 5:
            colors = (colors + [colors[-1]])[:5] if colors else default_cm[:]
            while len(colors) < 5:
                colors = [colors[0]] + colors + [colors[-1]]
                colors = colors[:5]
        elif len(colors) > 5:
            colors = [colors[0], colors[1], colors[len(colors)//2], colors[-2], colors[-1]]
        cmap = LinearSegmentedColormap.from_list("custom5", colors, N=256)

        # 4) Safe figure size
        fig_w = fig_h = None
        fs = s.get("figure_size", None)
        if isinstance(fs, (list, tuple)) and len(fs) >= 2:
            try:
                fig_w, fig_h = float(fs[0]), float(fs[1])
            except Exception:
                fig_w = fig_h = None
        if fig_w is None or fig_h is None:
            scale = s.get("figure_size_scale", 0.6) or 0.6
            try:
                scale = float(scale)
            except Exception:
                scale = 0.6
            # make sure len(...) never gets 0
            ncols = max(1, len(cols))
            nrows = max(1, len(rows))
            fig_w = max(8.0, ncols * scale)
            fig_h = max(4.0, nrows * scale)

        # 5) Safe limits / options
        vcenter_zero = bool(s.get("vcenter_zero", True))
        vmax_abs = s.get("vmax_abs", None)
        try:
            vmax_abs = float(vmax_abs)
            if vmax_abs <= 0:
                vmax_abs = None
        except (TypeError, ValueError):
            vmax_abs = None

        data = self.correlation_matrix.values.astype(float)
        absm = np.nanmax(np.abs(data)) if vmax_abs is None else float(vmax_abs)
        if not np.isfinite(absm) or absm == 0:
            absm = 1e-6

        fig, ax = plt.subplots(figsize=(fig_w, fig_h))
        if vcenter_zero:
            norm = TwoSlopeNorm(vmin=-absm, vcenter=0.0, vmax=absm)
            im = ax.imshow(data, aspect='auto', cmap=cmap, norm=norm)
        else:
            im = ax.imshow(data, aspect='auto', cmap=cmap, vmin=-absm, vmax=absm)

        ax.set_xticks(np.arange(len(cols)))
        rot = s.get("rotate_xticks", 45)
        try:
            rot = float(rot)
        except Exception:
            rot = 45.0
        ax.set_xticklabels(
            [self._pretty(c) for c in cols],
            rotation=rot, ha='right',
            fontsize=int(s.get("tick_fontsize", 9))  # <<< NEW
        )

        ax.set_yticks(np.arange(len(rows)))
        ax.set_yticklabels(rows, fontsize=int(s.get("tick_fontsize", 9))) 
        ax.set_title(self.figure_title, fontsize=s.get("title_fontsize", 14), fontweight='bold', pad=20)

        # annotations
        if bool(s.get("annotate", s.get("show_values", True))):
            val_fs = int(s.get("value_fontsize", 7) or 7)
            cap = absm
            for i in range(data.shape[0]):
                for j in range(data.shape[1]):
                    val = data[i, j]
                    if np.isfinite(val):
                        mag = abs(val)/(cap+1e-12)
                        color = 'white' if mag>0.6 else 'black' if mag>0.4 else '#333333'
                        ax.text(j, i, f'{val:.2f}', ha="center", va="center",
                                fontsize=val_fs, fontweight='bold', color=color)

        cbar = plt.colorbar(im, ax=ax, shrink=0.85)
        cbar.ax.set_ylabel(t_plot("colorbar"), rotation=90)

        if bool(s.get("grid_minor", True)):
            ax.set_xticks(np.arange(len(cols)+1)-0.5, minor=True)
            ax.set_yticks(np.arange(len(rows)+1)-0.5, minor=True)
            ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.3, alpha=0.3)

        plt.tight_layout()
        plt.show()
        print("✅ " + t("hm_done"))
        self._last_heatmap = (fig, ax)
        return fig, ax


    def _legend_anchor_from_pad(self, loc: str, pad: float):
        loc = (loc or "").lower()
        if loc == "lower center": return (0.5, -pad)
        if loc == "upper center": return (0.5, 1 + pad)
        if loc == "center left":  return (-pad, 0.5)
        if loc == "center right": return (1 + pad, 0.5)
        if loc == "lower left":   return (-pad, -pad)
        if loc == "lower right":  return (1 + pad, -pad)
        if loc == "upper left":   return (-pad, 1 + pad)
        if loc == "upper right":  return (1 + pad, 1 + pad)
        return None

    def plot_pie_network(self):
        if self.correlation_matrix is None: return
        print("\n🎨 " + t("net_creating"))
        s = self.network_settings

        G = nx.Graph()
        treatments = list(self.correlation_matrix.index)
        outcomes = list(self.correlation_matrix.columns)
        for t0 in treatments: G.add_node(t0, group='treatment')
        for o in outcomes:    G.add_node(o, group='outcome')

        edges_added = 0
        for t0 in treatments:
            row = self.correlation_matrix.loc[t0].dropna()
            row_abs = row.abs().sort_values(ascending=False)
            count = 0
            for o, abs_r in row_abs.items():
                if abs_r >= self.correlation_threshold and count < self.top_k:
                    r = row[o]
                    G.add_edge(t0, o, weight=abs_r, correlation=r)
                    edges_added += 1; count += 1
        if edges_added == 0:
            print("⚠ " + t("no_edges", thr=self.correlation_threshold)); return None

        radius = s["circle_radius"]; fig_size = s["figure_size"]
        fig, ax = plt.subplots(figsize=fig_size, constrained_layout=False)
        if s.get("background") != "white":
            fig.patch.set_facecolor(s["background"]); ax.set_facecolor(s["background"])

        pos = {}
        n_t = len(treatments)
        def _arc_linspace_deg(start_deg, end_deg, n):
            sdeg = float(s.get(start_deg, -84.0))
            edeg = float(s.get(end_deg,  84.0))
            # allow wrapping if user drags start past end
            if sdeg >= edeg:
                edeg += 360.0
            return np.deg2rad(np.linspace(sdeg, edeg, n, endpoint=True))

        theta_t = _arc_linspace_deg("treat_start_deg", "treat_end_deg", n_t)
        for i, t0 in enumerate(treatments):
            pos[t0] = (radius*np.cos(theta_t[i]), radius*np.sin(theta_t[i]))

        outcome_scores = []
        for o in outcomes:
            ang, w = [], []
            for t0 in treatments:
                if G.has_edge(t0,o):
                    idx = treatments.index(t0)
                    ang.append(theta_t[idx]); w.append(G.edges[t0,o]['weight'])
            if w:
                score = np.arctan2(np.average(np.sin(ang), weights=w),
                                   np.average(np.cos(ang), weights=w))
            else:
                score = np.pi
            outcome_scores.append((o, score))
        outcome_sorted = [o for o,_ in sorted(outcome_scores, key=lambda x: x[1], reverse=True)]
        n_o = len(outcome_sorted)
        theta_o = _arc_linspace_deg("out_start_deg", "out_end_deg", n_o)
        for i, o in enumerate(outcome_sorted):
            pos[o] = (radius*np.cos(theta_o[i]), radius*np.sin(theta_o[i]))

        for (u,v) in G.edges():
            r = G.edges[u,v]['correlation']
            color = s["positive_edge"] if r>0 else s["negative_edge"]
            width = s["edge_base_width"] + s["edge_width_scale"] * G.edges[u,v]['weight']
            ua, va = np.arctan2(pos[u][1], pos[u][0]), np.arctan2(pos[v][1], pos[v][0])
            diff = abs(ua - va);  diff = 2*np.pi - diff if diff>np.pi else diff
            curv = 0.1 + s["edge_curvature"] * (diff / np.pi)
            connectionstyle = f"arc3,rad={curv if r>0 else -curv}"
            arrow = FancyArrowPatch(pos[u], pos[v], connectionstyle=connectionstyle,
                                    arrowstyle='-', color=color, linewidth=width,
                                    alpha=s["edge_alpha"], zorder=1)
            ax.add_patch(arrow)

        node_sizes = {}
        for node in G.nodes():
            n_conn = len(list(G.edges(node)))
            node_sizes[node] = s["node_base_size"] + n_conn * s["node_size_increment"]

        for node in G.nodes():
            color = s["treatment_color"] if G.nodes[node]['group']=='treatment' else s["outcome_color"]
            ax.scatter(pos[node][0], pos[node][1], s=node_sizes[node],
                       c=color, edgecolors='white', linewidth=2, zorder=5)

        for node in G.nodes():
            x, y = pos[node]; r = np.hypot(x, y)
            if r > 0:
                ux, uy = x/r, y/r
                lx, ly = x + s["label_distance"]*ux, y + s["label_distance"]*uy
                ha = 'left' if x > 0.1 else 'right' if x < -0.1 else 'center'
                display_text = self._pretty(node) if G.nodes[node]['group']=='outcome' else node
                ax.text(lx, ly, display_text, fontsize=s["label_fontsize"], ha=ha, va='center')

        ax.set_aspect('equal')
        lim = radius + s["label_distance"] + 0.5
        ax.set_xlim(-lim, lim); ax.set_ylim(-lim, lim); ax.axis('off')

        if s["show_title"]:
            ax.text(
                0.5, 1.0 + float(s.get("title_pad_frac", -0.05)),
                t_plot("net_title", thr=self.correlation_threshold, k=self.top_k),
                transform=ax.transAxes, ha='center', va='bottom',
                fontsize=s["title_fontsize"], fontweight='bold', zorder=999, clip_on=False
            )


        if s["show_legend"]:
            treatment_marker = Line2D([0],[0], marker='o', color='w',
                                      markerfacecolor=s["treatment_color"], markersize=12,
                                      markeredgecolor='white', markeredgewidth=1.5,
                                      label=t_plot("legend_trt"), linestyle='')
            outcome_marker   = Line2D([0],[0], marker='o', color='w',
                                      markerfacecolor=s["outcome_color"], markersize=12,
                                      markeredgecolor='white', markeredgewidth=1.5,
                                      label=t_plot("legend_out"), linestyle='')
            pos_line = Line2D([0],[0], color=s["positive_edge"], linewidth=3, label=t_plot("legend_pos"))
            neg_line = Line2D([0],[0], color=s["negative_edge"], linewidth=3, label=t_plot("legend_neg"))
            legend_elems = [treatment_marker, outcome_marker, pos_line, neg_line]

            loc   = s.get("legend_position", "lower center")
            ncol  = int(s.get("legend_ncol", 1))
            pad   = float(s.get("legend_pad", -0.07))
            anchor = self._legend_anchor_from_pad(loc, pad)
            kw = dict(frameon=False, ncol=ncol, prop={'size': s["legend_fontsize"]})
            if anchor is not None:
                kw["loc"] = "center"; kw["bbox_to_anchor"] = anchor
            else:
                kw["loc"] = loc
            ax.legend(handles=legend_elems, **kw)

        plt.show()
        print(f"✅ {t('net_done', E=G.number_of_edges())}")
        self._last_network = (fig, ax)
        return fig, ax

    def get_significant_correlations(self):
        if self.correlation_matrix is None: return None
        th = self.correlation_threshold
        print(f"\n🔍 {t('sig_header', thr=th)}\n" + "-"*70)
        rows = []
        for t0 in self.correlation_matrix.index:
            for o in self.correlation_matrix.columns:
                r = float(self.correlation_matrix.loc[t0,o])
                if np.isfinite(r) and abs(r)>th:
                    rows.append({"Treatment":t0, "Outcome":o, "r":r, "abs_r":abs(r)})
        if rows:
            df = pd.DataFrame(rows).sort_values("abs_r", ascending=False)
            for _,row in df.head(20).iterrows():
                arrow = "↑" if row["r"]>0 else "↓"
                print(f"  {row['Treatment']:<30} → {row['Outcome']:<15} r = {row['r']:+.3f} {arrow}")
            if len(df)>20:
                print(f"\n  ... {len(df)-20} more.")
            return df
        else:
            print("  " + t("none")); return None

    def save_last_figure(self, kind='heatmap', filename=None, dpi=300, transparent=False):
        pair = {'heatmap': self._last_heatmap, 'network': self._last_network}.get(kind)
        if pair is None:
            print(f"⚠️ No {kind} figure yet."); return None
        fig, _ = pair
        base = os.path.splitext(os.path.basename(self.data_file))[0]
        if not filename: filename = f"{base}_{kind}.png"
        fig.savefig(filename, dpi=int(dpi), bbox_inches='tight', transparent=bool(transparent))
        print(f"💾 {t('saved_to')} : {filename}")
        return filename

    def run_full_analysis(self, show_plots: bool = False):
        print("\n" + "="*80); print(f"   {t('start')}"); print("="*80)
        if self.load_and_detect() is None:
            print("\n❌ " + t("no_numeric")); return None
        self.prepare_design_matrix()
        self.calculate_correlations()
        self.save_results()

        if show_plots:
            print("\n" + "="*80); print(f"   {t('viz_header')}"); print("="*80)
            print("\n📊 " + t("viz1")); self.plot_heatmap()
            print("\n📊 " + t("viz2")); self.plot_pie_network()

        print("\n" + "="*80); print(f"   {t('top_header')}"); print("="*80)
        self.get_significant_correlations()
        print("\n" + "="*80); print(f"   ✨ {t('complete')} ✨"); print("="*80)
        print(f"\n 📁 {t('saved_to')} : {self.output_csv}")
        print(f" 📊 {t('summary_data')}: {self.data_file}")
        print(f" 🧪 {t('summary_design')}: {len(self.material_cols)} × {len(self.additive_cols)}")
        print(f" 📈 {t('summary_outcomes')}: {len(self.outcome_cols)}")
        return self

# =============================================================================
# MAIN
# =============================================================================
if __name__ == "__main__":
    print(t("welcome_banner"))
    print(SAMPLE_FORMAT)

    print(f"\n📂 {t('using_file')}: {MY_EXCEL_FILE}")
    print(f"🎯 {t('thr')}: {SHOW_CORRELATIONS_ABOVE}")
    print(f"🔝 {t('topk')}: {TOP_CORRELATIONS_PER_TREATMENT}")

    try:
        analyzer = AutoCorrelationAnalyzer(
            excel_file=MY_EXCEL_FILE,
            title=MY_ANALYSIS_TITLE,
            correlation_threshold=SHOW_CORRELATIONS_ABOVE,
            top_k=TOP_CORRELATIONS_PER_TREATMENT,
            network_settings=NETWORK_SETTINGS,
            heatmap_settings=HEATMAP_SETTINGS
        )
        analyzer.run_full_analysis(show_plots=False)

        # ==== Heatmap UI (Chinese labels) ====
        if _running_in_notebook():
            print("\n🧩 " + t("heatmap_controls"))
            cp_layout = widgets.Layout(width='200px')
            cp_style  = {'description_width': '90px'}

            c1 = widgets.ColorPicker(value=HEATMAP_SETTINGS["colormap"][0], description=t('c1'),
                                     layout=cp_layout, style=cp_style)
            c2 = widgets.ColorPicker(value=HEATMAP_SETTINGS["colormap"][1], description='c2',
                                     layout=cp_layout, style=cp_style)
            c3 = widgets.ColorPicker(value=HEATMAP_SETTINGS["colormap"][2], description=t('c3'),
                                     layout=cp_layout, style=cp_style)
            c4 = widgets.ColorPicker(value=HEATMAP_SETTINGS["colormap"][3], description='c4',
                                     layout=cp_layout, style=cp_style)
            c5 = widgets.ColorPicker(value=HEATMAP_SETTINGS["colormap"][4], description=t('c5'),
                                     layout=cp_layout, style=cp_style)

            hm_w = widgets.FloatSlider(value=12.0, min=6, max=24, step=0.5, description=t('width_in'))
            hm_h = widgets.FloatSlider(value=8.0,  min=4, max=24, step=0.5, description=t('height_in'))
            use_explicit_size = widgets.Checkbox(value=True, description=t_ui('use_explicit'))

            annotate_chk = widgets.Checkbox(value=HEATMAP_SETTINGS.get("show_values", True), description=t_ui('annotate'))
            rotate_deg   = widgets.FloatSlider(value=45, min=0, max=90, step=1, description=t('xrot'))
            vcenter_zero = widgets.Checkbox(value=True, description=t_ui('center0'))
            vmax_abs     = widgets.FloatText(value=None, description=t_ui('vmax_abs'))
            grid_minor   = widgets.Checkbox(value=True, description=t_ui('grid_minor'))

            redraw_heatmap_btn = widgets.Button(description=t('redraw_heatmap'), button_style='')
            out_hm = widgets.Output()

            def _redraw_heatmap(_=None):
                with out_hm:
                    clear_output(wait=True)
                    analyzer.heatmap_settings["colormap"] = [c1.value, c2.value, c3.value, c4.value, c5.value]
                    analyzer.heatmap_settings["rotate_xticks"] = float(rotate_deg.value)
                    analyzer.heatmap_settings["annotate"] = bool(annotate_chk.value)
                    analyzer.heatmap_settings["show_values"] = bool(annotate_chk.value)
                    analyzer.heatmap_settings["vcenter_zero"] = bool(vcenter_zero.value)
                    analyzer.heatmap_settings["grid_minor"] = bool(grid_minor.value)

                    # NEW: pass font sizes into settings
                    analyzer.heatmap_settings["tick_fontsize"]  = int(tick_fs.value)
                    analyzer.heatmap_settings["value_fontsize"] = int(cell_fs.value)

                    if use_explicit_size.value:
                        analyzer.heatmap_settings["figure_size"] = (float(hm_w.value), float(hm_h.value))
                    else:
                        analyzer.heatmap_settings["figure_size"] = None

                    v = vmax_abs.value
                    try: v = float(v)
                    except (TypeError, ValueError): v = None
                    analyzer.heatmap_settings["vmax_abs"] = None if (v is None or v <= 0) else v

                    print("🎨 " + t("drawing_hm"))
                    analyzer.plot_heatmap()

            redraw_heatmap_btn.on_click(_redraw_heatmap)
            save_name_hm = widgets.Text(value='', placeholder='auto name', description=t('filename'))
            save_dpi_hm = widgets.IntSlider(value=300, min=72, max=600, step=12, description=t('dpi'))
            tick_fs = widgets.IntSlider(
                value=int(HEATMAP_SETTINGS.get("tick_fontsize", 9)),
                min=6, max=24, step=1, description=t('tick fs')
            )
            cell_fs = widgets.IntSlider(
                value=int(HEATMAP_SETTINGS.get("value_fontsize", 7)),
                min=6, max=24, step=1, description=t('cell fs')
            )
            save_trans_hm = widgets.Checkbox(value=False, description=t('transparent'))
            save_heatmap_btn = widgets.Button(description=t('save_hm'), button_style='success')

            def _save_heatmap(_=None):
                fname = save_name_hm.value.strip() or None
                analyzer.save_last_figure('heatmap', filename=fname, dpi=int(save_dpi_hm.value),
                                          transparent=bool(save_trans_hm.value))
            save_heatmap_btn.on_click(_save_heatmap)

            ui_hm = widgets.VBox([
                widgets.HTML(f"<b>{t('heatmap_colors')}</b>"),
                widgets.HBox([c1, c2, c3, c4, c5]),
                widgets.HTML(f"<b>{t('size')}</b>"),
                widgets.HBox([use_explicit_size, hm_w, hm_h]),
                widgets.HTML(f"<b>{t('display')}</b>"),
                widgets.HBox([annotate_chk, grid_minor]),
                widgets.HBox([vcenter_zero, rotate_deg, vmax_abs]),
                widgets.HBox([tick_fs, cell_fs]),   # <<< NEW row for font sliders
                redraw_heatmap_btn,
                widgets.HTML(f"<b>{t('save')}</b>"),
                widgets.HBox([save_name_hm, save_dpi_hm, save_trans_hm]),
                save_heatmap_btn
            ])

            display(ui_hm, out_hm)
            if analyzer.correlation_matrix is None or analyzer.correlation_matrix.empty:
                analyzer.calculate_correlations()
            _redraw_heatmap()

        # ==== Network UI (Chinese labels) ====
        if _running_in_notebook():
            print("\n🎛️ " + t("net_controls_tip"))
            save_name_net = widgets.Text(value='', placeholder='auto name', description=t('filename'))
            save_dpi_net = widgets.IntSlider(value=300, min=72, max=600, step=12, description=t('dpi'))
            save_trans_net = widgets.Checkbox(value=False, description=t('transparent'))
            save_network_btn = widgets.Button(description=t('save_net'), button_style='success')
            tr_start = widgets.FloatSlider(
                value=analyzer.network_settings.get("treat_start_deg", -84.0),
                min=-180, max=180, step=1, description="trt start°", continuous_update=False)
            tr_end = widgets.FloatSlider(
                value=analyzer.network_settings.get("treat_end_deg", 84.0),
                min=-180, max=180, step=1, description="trt end°", continuous_update=False)
            out_start = widgets.FloatSlider(
                value=analyzer.network_settings.get("out_start_deg", 96.0),
                min=-180, max=540, step=1, description="out start°", continuous_update=False)
            out_end = widgets.FloatSlider(
                value=analyzer.network_settings.get("out_end_deg", 264.0),
                min=-180, max=540, step=1, description="out end°", continuous_update=False)

            def _save_network(_=None):
                fname = save_name_net.value.strip() or None
                analyzer.save_last_figure('network', filename=fname, dpi=int(save_dpi_net.value),
                                          transparent=bool(save_trans_net.value))
            save_network_btn.on_click(_save_network)

            if analyzer.correlation_matrix is None:
                analyzer.calculate_correlations()

            label_slider = widgets.FloatSlider(value=analyzer.network_settings.get("label_distance", 0.15),
                                               min=0.02, max=0.80, step=0.01, description=t('label_dist'),
                                               continuous_update=False)
            radius_slider = widgets.FloatSlider(value=analyzer.network_settings.get("circle_radius", 1.3),
                                                min=0.6, max=2.0, step=0.05, description=t('circle_radius'),
                                                continuous_update=False)
            curve_slider = widgets.FloatSlider(value=analyzer.network_settings.get("edge_curvature", 0.3),
                                               min=0.0, max=0.7, step=0.02, description=t('edge_curv'),
                                               continuous_update=False)
            thresh_slider = widgets.FloatSlider(value=float(analyzer.correlation_threshold),
                                                min=0.0, max=0.9, step=0.01, description=t('r_thresh'),
                                                continuous_update=False)
            topk_slider = widgets.IntSlider(value=int(analyzer.top_k), min=1, max=20, step=1,
                                            description=t('top_per_trt'), continuous_update=False)
            redraw_btn = widgets.Button(description=t('redraw'), button_style='primary')
            out = widgets.Output()

            legend_pos_dropdown = widgets.Dropdown(
                options=["upper left","upper right","lower left","lower right","upper center","lower center",
                         "center left","center right","center"],
                value=analyzer.network_settings.get("legend_position","lower center"),
                description=t('legend_loc'),
            )
            legend_pad_slider = widgets.FloatSlider(value=float(analyzer.network_settings.get("legend_pad", -0.07)),
                                                    min=-0.2, max=0.3, step=0.002,
                                                    description=t('legend_pad'), continuous_update=False)
            legend_ncol = widgets.IntSlider(value=int(analyzer.network_settings.get("legend_ncol", 2)),
                                            min=1, max=4, step=1, description=t('legend_cols'),
                                            continuous_update=False)
            title_pad_frac_slider = widgets.FloatSlider(
                value=float(analyzer.network_settings.get("title_pad_frac", -0.05)),
                min=-0.15, max=0.30, step=0.002, description=t('title_pad_frac'), continuous_update=False
            )

            def _redraw(_=None):
                with out:
                    clear_output(wait=True)
                    analyzer.network_settings["label_distance"] = float(label_slider.value)
                    analyzer.network_settings["circle_radius"]  = float(radius_slider.value)
                    analyzer.network_settings["edge_curvature"] = float(curve_slider.value)
                    analyzer.correlation_threshold              = float(thresh_slider.value)
                    analyzer.top_k                               = int(topk_slider.value)
                    analyzer.network_settings["legend_position"] = legend_pos_dropdown.value
                    analyzer.network_settings["legend_ncol"]     = int(legend_ncol.value)
                    analyzer.network_settings["legend_pad"]      = float(legend_pad_slider.value)
                    analyzer.network_settings["title_pad_frac"]  = float(title_pad_frac_slider.value)
                    analyzer.network_settings["treat_start_deg"] = float(tr_start.value)
                    analyzer.network_settings["treat_end_deg"]   = float(tr_end.value)
                    analyzer.network_settings["out_start_deg"]   = float(out_start.value)
                    analyzer.network_settings["out_end_deg"]     = float(out_end.value)

                    print("🎨 " + t("drawing_net"))
                    fig_ax = analyzer.plot_pie_network()
                    if fig_ax is None:
                        print(t("no_edges", thr=analyzer.correlation_threshold))

            redraw_btn.on_click(_redraw)
            ui = widgets.VBox([
                widgets.HBox([label_slider, radius_slider, curve_slider]),
                widgets.HBox([tr_start, tr_end, out_start, out_end]),   # ← new angle tuners
                widgets.HBox([thresh_slider, topk_slider]),
                widgets.HBox([legend_pos_dropdown, legend_ncol, legend_pad_slider]),
                widgets.HBox([title_pad_frac_slider]),
                redraw_btn,
                widgets.HTML(f"<b>{t('save')}</b>"),
                widgets.HBox([save_name_net, save_dpi_net, save_trans_net]),
                save_network_btn
            ])

            display(ui, out); _redraw()

    except Exception as e:
        print(f"\n❌ {e}")
        print("\n💡 常见排查：\n  1) 确认 Excel 文件存在且可读。\n  2) 检查文件名/扩展名（.xlsx/.xls）。\n  3) 二元列应为 0/1，结果列应为数值。")



╔════════════════════════════════════════════════════════════════════╗
║   📊 欢迎使用相关性分析工具                                        ║
╚════════════════════════════════════════════════════════════════════╝


上传前请按下述英文表头整理（仅 ASCII），示例：

index | AlkaliN | AvailP | AvailK | SOM  | pH  | AvailZn | Yield
------+---------+--------+--------+------+-----+---------+-------
Zn0   |  98.4   |  19.4  |  172.4 | 14.8 | 6.11|  6.74   | 10257.1
LZn   |  65.7   |  33.0  |  189.7 | 15.0 | 5.34|  7.96   | 12426.8
HZn   |  85.5   |  20.7  |  176.7 | 13.8 | 5.12|  8.05   | 12522.5

上传规范：
- 第一列表头必须为 index；行值仅能是 Zn0、LZn、HZn（锌的 3 个水平）。
- 其它列为数值型结果变量（英文/数字/下划线，勿用中文或全角字符）。
- 小数使用点号 .（如 7.83），不要逗号。
- 可自行增删结果列；顺序不限。
- 保存为 .xlsx/.xls 后上传。


📂 使用的 Excel 文件: gao_pca.xlsx
🎯 |r| 阈值: 0.3
🔝 每个处理的 Top 数: 6

   开始自动相关性分析

✅ 已从文件读取 3 行： 'gao_pca.xlsx'

🧹 正在清理数据…

🔎 检测到可能的单因子标签： Zn0, LZn, HZn


是否启用【单因子梯度】模式？(y/N)： 



🧩 检测到单因子梯度设计： ['HZn', 'LZn', 'Zn0']

🔍 （索引解析）自动识别的设计：
  📦 原料/材料: []
  🧪 添加剂: []
  📊 结果变量: ['Soil organic matter', 'Available Zn', 'Corn yield', 'FI', 'BIX'] ...

✅ 设计矩阵：3 个样本 × 3 个处理

📈 正在计算与 17 个结果变量的相关系数…
✅ 相关计算完成。

💾 结果已保存至 : gao_pca_correlation_results.csv

   Top 相关结果

🔍 显著相关（|r| > 0.3）：
----------------------------------------------------------------------
  Zn0                            → CHA             r = -1.000 ↓
  Zn0                            → Corn yield      r = -0.999 ↓
  Zn0                            → Available Zn    r = -0.998 ↓
  Zn0                            → O/C ratio of HA r = +0.996 ↑
  HZn                            → Exothermic ratio of high to moderate temperature r = +0.993 ↑
  Zn0                            → CWSS            r = -0.991 ↓
  HZn                            → Soil organic matter r = -0.988 ↓
  HZn                            → CHU             r = -0.987 ↓
  HZn                            → C/N ratio of HA r = -0.987 ↓
  LZn                

Output()


🎛️ 使用下面的滑块微调网络图布局。


Output()